У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression



from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [66]:
customer_df = pd.read_csv("drive/MyDrive/machine_learning_for_people/HW.2.2/customer_segmentation_train.csv", index_col=0)

In [67]:
customer_df.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [68]:
customer_df = customer_df.copy()


In [69]:
customer_df.shape

(8068, 10)

In [70]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 693.3+ KB


In [71]:
null_count = customer_df.isnull().sum()

In [72]:
null_count

,0
Gender,0
Ever_Married,140
Age,0
Graduated,78
Profession,124
Work_Experience,829
Spending_Score,0
Family_Size,335
Var_1,76
Segmentation,0


In [73]:
null_percentage = round((customer_df.isnull().sum()/customer_df.shape[0])*100, 2)

In [74]:
null_percentage

,0
Gender,0.00
Ever_Married,1.74
Age,0.00
Graduated,0.97
Profession,1.54
Work_Experience,10.28
Spending_Score,0.00
Family_Size,4.15
Var_1,0.94
Segmentation,0.00


В нашому наборі даних кількість записів 8068, кількість змінних 10.

Числові (int64, float64) – 3 стовпці.
Категоріальні (object) – 7 стовпців.

Найбільше пропущених значень в колонці Work_Experience 10.28%.

Колонка Family_Size має 4.15% пропущених значень, Ever_Married	1.74%, Profession	1.54%, Var_1	0.94%, Graduated	0.97%.



In [75]:
# Поглянемо які унікальні значення містять категоріальні колонки
unique_values = {col: customer_df[col].unique() for col in customer_df.select_dtypes(include='object').columns}
unique_values

{'Gender': array(['Male', 'Female'], dtype=object),
 'Ever_Married': array(['No', 'Yes', nan], dtype=object),
 'Graduated': array(['No', 'Yes', nan], dtype=object),
 'Profession': array(['Healthcare', 'Engineer', 'Lawyer', 'Entertainment', 'Artist',
        'Executive', 'Doctor', 'Homemaker', 'Marketing', nan], dtype=object),
 'Spending_Score': array(['Low', 'Average', 'High'], dtype=object),
 'Var_1': array(['Cat_4', 'Cat_6', 'Cat_7', 'Cat_3', 'Cat_1', 'Cat_2', nan, 'Cat_5'],
       dtype=object),
 'Segmentation': array(['D', 'A', 'B', 'C'], dtype=object)}

In [76]:
# Визначення числових колонок для ознак
numeric_cols = customer_df.select_dtypes(include=[np.number]).columns.tolist()

# Визначення категоріальних колонок для ознак
categorical_cols = customer_df.select_dtypes(include=['object']).columns.tolist()

print("Числові колонки:", numeric_cols)
print("Категоріальні колонки:", categorical_cols)

Числові колонки: ['Age', 'Work_Experience', 'Family_Size']
Категоріальні колонки: ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'Segmentation']


Щоб зрозуміти як заповнити пропущені значення проведемо деякий аналіз колонок з пропущеними значеннями

In [77]:
print(customer_df['Work_Experience'].value_counts(),'\n', customer_df['Family_Size'].value_counts())

Work_Experience
1.0     2354
0.0     2318
9.0      474
8.0      463
2.0      286
3.0      255
4.0      253
6.0      204
7.0      196
5.0      194
10.0      53
11.0      50
12.0      48
13.0      46
14.0      45
Name: count, dtype: int64 
 Family_Size
2.0    2390
3.0    1497
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: count, dtype: int64


In [78]:
customer_df[['Work_Experience', 'Family_Size']].describe()

,Work_Experience,Family_Size
count,7239.000000,7733.000000
mean,2.641663,2.850123
std,3.406763,1.531413
min,0.000000,1.000000
25%,0.000000,2.000000
50%,1.000000,3.000000
75%,4.000000,4.000000
max,14.000000,9.000000


In [79]:
print(customer_df['Ever_Married'].value_counts(),'\n', customer_df['Graduated'].value_counts())

Ever_Married
Yes    4643
No     3285
Name: count, dtype: int64 
 Graduated
Yes    4968
No     3022
Name: count, dtype: int64


In [80]:
print(customer_df['Segmentation'].value_counts())

Segmentation
D    2268
A    1972
C    1970
B    1858
Name: count, dtype: int64


In [81]:
print(customer_df['Profession'].value_counts(),'\n', customer_df['Var_1'].value_counts())

Profession
Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: count, dtype: int64 
 Var_1
Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: count, dtype: int64


In [82]:
from sklearn.impute import SimpleImputer

Ever_Married (1.74% пропусків) і Graduated (0.97% пропусків). Це бінарні категоріальні змінні зі значеннями ("Yes" або "No"). Заповнимо пропущенні значення найбільш поширеним значенням (модою).

In [83]:
categorical_imputer_mode = SimpleImputer(strategy='most_frequent')
customer_df[['Ever_Married', 'Graduated']] = categorical_imputer_mode.fit_transform(customer_df[['Ever_Married', 'Graduated']])

Пропущенні значення в категоріальних колонках 'Profession', 'Var_1' заповнимо Unknown

In [84]:
customer_df[['Profession', 'Var_1']] = customer_df[['Profession', 'Var_1']].fillna('Unknown')

Заповнення числових колонок

Пропущені значення в колонці 'Work_Experience' становлять 10.28%. Пропуски досить значні, тому можна ввести окрему категорію. Замінимо пропуски числом, що не є в діапазоні, наприклад -1.

In [85]:
customer_df['Work_Experience'] = customer_df['Work_Experience'].fillna(-1)

Family_Size (4.15% пропусків). Заповнимо пропущенні значення 'Family_Size' медіаною

In [86]:
customer_df['Family_Size'] = customer_df['Family_Size'].fillna(customer_df['Family_Size'].median())

In [87]:
null_count = customer_df.isnull().sum()

In [89]:
print(null_count)

Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64


Тепер пропущених значень в датасеті немає.

In [90]:
customer_df1 = customer_df.copy() #для наступної задачі

Закодуємо категоріальні колонки, щоб модель могла прийняти на вхід числові значення

Категоріальну змінну Spending_Score можна закодувати через OrdinalEncoder, оскільки є сенс у порядку.
Інші категоріальні змінні закодуємо за допомогою LabelEncode.

In [91]:
from sklearn.preprocessing import OrdinalEncoder

In [92]:
# Кодування колонки'Spending_Score'
spending_score_encoded = OrdinalEncoder(categories=[['Low', 'Average', 'High']])
customer_df['Spending_Score'] = spending_score_encoded.fit_transform(customer_df[['Spending_Score']])

In [93]:
from sklearn import preprocessing

# Ініціалізуємо LabelEncoder
le = preprocessing.LabelEncoder()

# Список стовпців для кодування
columns_to_encode = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Var_1', 'Segmentation']

# Кодуємо кожен стовпець в циклі
for column in columns_to_encode:
    customer_df[column] = le.fit_transform(customer_df[column])

In [94]:
# Відокремлюємо ознаки X та цільову змінну y
X = customer_df.drop(columns=['Segmentation'])
y = customer_df['Segmentation']

In [95]:
# Скейлимо вхідні дані
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [96]:
# Розбиття даних на тренувальний і тестовий набір
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [98]:
# Ініціалізація та навчання моделі
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [99]:
# Прогнозування
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [100]:
# Оцінка результатів
print("\nОцінка на тренувальному наборі:")
print(classification_report(y_train, y_train_pred))

print("\nОцінка на тестувальному наборі:")
print(classification_report(y_test, y_test_pred))


Оцінка на тренувальному наборі:
              precision    recall  f1-score   support

           0       0.42      0.42      0.42      1581
           1       0.35      0.13      0.19      1489
           2       0.47      0.64      0.54      1590
           3       0.62      0.73      0.67      1794

    accuracy                           0.49      6454
   macro avg       0.46      0.48      0.45      6454
weighted avg       0.47      0.49      0.47      6454


Оцінка на тестувальному наборі:
              precision    recall  f1-score   support

           0       0.41      0.40      0.40       391
           1       0.38      0.14      0.20       369
           2       0.43      0.62      0.50       380
           3       0.62      0.72      0.67       474

    accuracy                           0.48      1614
   macro avg       0.46      0.47      0.44      1614
weighted avg       0.47      0.48      0.46      1614



Підготовлені дані ми передали в модель логістичної регресії. Провели навчання моделі на тренувальному наборі, зробили передбачення на тренувальному та тестовому наборі даних.

Загальний аналіз
Accuracy 0.49 (train) / 0.48 (test)

Це означає, що модель правильно класифікує приблизно половину прикладів.

Precision = скільки з передбачених позитивів правильні

Найкраще модель працює на класі 3 (0.62 train / 0.62 test).
Найгірше на класі 1 (0.35 train / 0.38 test).

Recall = скільки з реальних позитивів правильно знайдені

Модель найкраще знаходить приклади класу 3 (73% train / 72% test).
Клас 1 має дуже низький recall (13% train / 14% test) – значить, модель погано знаходить приклади цього класу.

F1-score (Середнє Precision та Recall)

Найкращий F1-score для класу 3 (0.67 train / 0.67 test).
Найгірший для класу 1 (0.19 train / 0.20 test).

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [101]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)


Застосуємо метод оверсемплінгу даних SMOTENC

In [103]:
# Визначення категоріальних колонок для ознак
categorical_cols = customer_df1.select_dtypes(include=['object']).columns.drop('Segmentation').tolist()


In [104]:
categorical_cols

['Gender',
 'Ever_Married',
 'Graduated',
 'Profession',
 'Spending_Score',
 'Var_1']

In [105]:
# Індекси категоріальних колонок
categorical_indices = [customer_df.columns.get_loc(col) for col in categorical_cols]

In [106]:
categorical_indices

[0, 1, 3, 4, 6, 8]

Передамо в метод оверсемплігу даних SMOTENC індекси колонок, які вважаємо категоріальними для того, щоб метод обробляв числові колонки як числові, а  категоріальні, індекси яких ми передали, як категоріальні.

In [107]:
smotenc = SMOTENC(categorical_features=categorical_indices,
                  random_state=42,
                  sampling_strategy='auto')

# Застосовуємо SMOTENC до тренувальних даних
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)


In [108]:
print(f"Оригінальний X_train: {X_train.shape}")
print(f"Після SMOTENC X_train_smotenc: {X_train_smotenc.shape}")

Оригінальний X_train: (6454, 9)
Після SMOTENC X_train_smotenc: (7256, 9)


Оригінальний X_train: (6454, 9)
Після SMOTENC X_train_smotenc: (7256, 9)
Ми бачимо, що SMOTENC згенерував нам синтетичні зразки.

Передамо в модель логістичної регресії тренувальний набір даних збаласований за домопогою SMOTENC

In [109]:
model_smotenc = LogisticRegression(max_iter=1000, random_state=42)
model_smotenc.fit(X_train_smotenc, y_train_smotenc)

y_train_pred = model_smotenc.predict(X_train_smotenc)
y_test_pred = model_smotenc.predict(X_test)

print("\nОцінка на тестувальному наборі:")
print(classification_report(y_test, y_test_pred))


Оцінка на тестувальному наборі:
              precision    recall  f1-score   support

           0       0.42      0.39      0.41       394
           1       0.38      0.23      0.29       372
           2       0.49      0.61      0.54       394
           3       0.64      0.75      0.69       454

    accuracy                           0.51      1614
   macro avg       0.48      0.50      0.48      1614
weighted avg       0.49      0.51      0.49      1614



Модель показала дещо кращі метрики у порівнянні з результатами моделі без балансування класів

Застосуємо метод SMOTETomek

In [110]:
from imblearn.combine import SMOTETomek

smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train, y_train)

model_smotetomek = LogisticRegression(max_iter=1000, random_state=42)
model_smotetomek.fit(X_train_smotetomek, y_train_smotetomek)

LogisticRegression(max_iter=1000, random_state=42)

In [111]:
y_train_pred = model_smotetomek.predict(X_train_smotetomek)
y_test_pred =model_smotetomek.predict(X_test)

print("\nОцінка на тестувальному наборі:")
print(classification_report(y_test, y_test_pred))


Оцінка на тестувальному наборі:
              precision    recall  f1-score   support

           0       0.43      0.40      0.41       394
           1       0.38      0.23      0.28       372
           2       0.49      0.62      0.55       394
           3       0.64      0.75      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.49      0.51      0.49      1614



Метод  SMOTETomek одночасно додає синтетичні зразки меншості та видаляє потенційні "шумові" зразки більшості.

Модель логістичної регресії на даних збалансованих SMOTETomek показує кращу якість навчання і дає дещо кращі результати.

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [112]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

Логістична регресія зі стратегією one-vs-rest на оригінальних даних

In [114]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train, y_train)
ovr_predictions_original = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test, ovr_predictions_original))


              precision    recall  f1-score   support

           0       0.41      0.35      0.38       394
           1       0.44      0.11      0.18       372
           2       0.46      0.66      0.55       394
           3       0.58      0.79      0.67       454

    accuracy                           0.50      1614
   macro avg       0.47      0.48      0.44      1614
weighted avg       0.48      0.50      0.46      1614



Логістична регресія зі стратегією one-vs-rest на даних збалансованих з SMOTENC

In [115]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
ovr_model_smotenc = OneVsRestClassifier(log_reg)
ovr_model_smotenc.fit(X_train_smotenc, y_train_smotenc)
ovr_predictions_smotenc = ovr_model_smotenc.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test, ovr_predictions_smotenc))

              precision    recall  f1-score   support

           0       0.42      0.36      0.39       394
           1       0.38      0.19      0.25       372
           2       0.47      0.62      0.53       394
           3       0.61      0.77      0.68       454

    accuracy                           0.50      1614
   macro avg       0.47      0.48      0.46      1614
weighted avg       0.48      0.50      0.47      1614



Логістична регресія зі стратегією one-vs-rest на даних збалансованих з SMOTETomek

In [116]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
ovr_model_smotetomek = OneVsRestClassifier(log_reg)
ovr_model_smotetomek.fit(X_train_smotetomek, y_train_smotetomek)
ovr_predictions_smotetomek = ovr_model_smotetomek.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test, ovr_predictions_smotetomek))

              precision    recall  f1-score   support

           0       0.42      0.39      0.40       394
           1       0.36      0.13      0.19       372
           2       0.46      0.63      0.53       394
           3       0.61      0.77      0.68       454

    accuracy                           0.50      1614
   macro avg       0.46      0.48      0.45      1614
weighted avg       0.47      0.50      0.46      1614



In [117]:
#Порівняння за F1-score
f1_original = f1_score(y_test, ovr_predictions_original, average='weighted')
f1_smotenc = f1_score(y_test, ovr_predictions_smotenc, average='weighted')
f1_smtomek = f1_score(y_test, ovr_predictions_smotetomek, average='weighted')

In [118]:
print(f1_original, f1_smotenc, f1_smtomek )

0.4556751101798187 0.47415139498601633 0.4642947996896739


In [119]:
# Усереднені метрики macro та micro для OvR
original_macro_precision = precision_score(y_test, ovr_predictions_original, average='macro')
original_micro_precision = precision_score(y_test, ovr_predictions_original, average='micro')
original_macro_recall = recall_score(y_test, ovr_predictions_original, average='macro')
original_micro_recall = recall_score(y_test, ovr_predictions_original, average='micro')

# Усереднені метрики macro та micro для OvR
smotenc_macro_precision = precision_score(y_test, ovr_predictions_smotenc, average='macro')
smotenc_micro_precision = precision_score(y_test, ovr_predictions_smotenc, average='micro')
smotenc_macro_recall = recall_score(y_test, ovr_predictions_smotenc, average='macro')
smotenc_micro_recall = recall_score(y_test, ovr_predictions_smotenc, average='micro')

# Усереднені метрики macro та micro для OvR
smotetomek_macro_precision = precision_score(y_test, ovr_predictions_smotetomek, average='macro')
smotetomek_micro_precision = precision_score(y_test, ovr_predictions_smotetomek, average='micro')
smotetomek_macro_recall = recall_score(y_test, ovr_predictions_smotetomek, average='macro')
smotetomek_micro_recall = recall_score(y_test, ovr_predictions_smotetomek, average='micro')



In [120]:
# Створимо датафрейм для відображення результатів
results = pd.DataFrame({
    'Metric': ['Macro Precision', 'Micro Precision', 'Macro Recall', 'Micro Recall'],
    'OvR_original': [original_macro_precision, original_micro_precision, original_macro_recall, original_micro_recall],
    'smotenc_original': [smotenc_macro_precision, smotenc_micro_precision, smotenc_macro_recall, smotenc_micro_recall],
    'smotetomek_original': [smotetomek_macro_precision, smotetomek_micro_precision, smotetomek_macro_recall, smotetomek_micro_recall]

})
print(results)

            Metric  OvR_original  smotenc_original  smotetomek_original
0  Macro Precision      0.474257          0.470565             0.461765
1  Micro Precision      0.496283          0.499380             0.495663
2     Macro Recall      0.479720          0.484577             0.480187
3     Micro Recall      0.496283          0.499380             0.495663


Висновок:

SMOTENC працює краще за SMOTETomek у більшості метрик, особливо у F1-score та Recall.


SMOTETomek не дав такого покращення, оскільки він не тільки додає дані, але й видаляє "шумові" дані і може видалити важливі зразки (особливо якщо даних мало).


Наш датасет має невисокий рівень дисбалансу класів, тому після застосування методів балансування (SMOTENC, SMOTETomek) результати не змінилися суттєво.



